# RAG Customer Support using GigaChat and FAISS

This notebook implements a **Retrieval-Augmented Generation (RAG)** pipeline for customer support automation.

### Components Overview

- **Language Model (LLM):** `GigaChat` — used to generate structured, human-like answers.
- **Embedding Model:** `sentence-transformers/paraphrase-multilingual-mpnet-base-v2` — generates dense vector representations of text.
- **Vector Database:** `FAISS` — used for efficient similarity search and context retrieval.

The pipeline retrieves relevant answers from a dataset based on the semantic similarity of embeddings, then uses **GigaChat** to generate a final, context-aware response.


## End-to-End RAG Flow Diagram

```markdown
User Query (text)
       │
       ▼
[Embedding Model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2]
       │
       ▼
Query Embedding (vector)
       │
       ▼
[FAISS Index Search]
       │
       ▼
Top-k Closest Contexts (text snippets from dataset)
       │
       ▼
Constructed Prompt:
──────────────────────────────────────────────
Context Information:
<retrieved context>

Example Interactions:
<few-shot examples>

Current Query:
<user's question>

Response Requirements:
<formatting & output JSON instructions>
──────────────────────────────────────────────
       │
       ▼
[LLM: GigaChat]
       │
       ▼
Generated Response (JSON: {response, confidence})
       │
       ▼
Postprocessing:
- Parse JSON
- Compute similarity (optional)
- Return structured output
```

### 💡 What GigaChat Actually Receives

GigaChat **does not** get embeddings directly.

It receives a **text prompt** that includes:
- the **original user query**
- the **retrieved text context** (not vectors)
- the **few-shot examples**
- and **explicit formatting & style instructions**


**1.  Install the necessary packages**

In [4]:
!pip install langchain
!pip install langchain-gigachat
!pip install langchain-community
!pip install faiss-gpu-cu12
!pip install scikit-learn
!pip install tqdm
!pip install fsspec 
!pip install sentence-transformers

In [5]:
import pandas as pd
import numpy as np
import json
import time
import faiss
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from kaggle_secrets import UserSecretsClient
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type, wait_random_exponential
import base64

**2. Set up API key**

To run the following cell, the `GigaChat API token` must be stored in a Kaggle secret named `GIGACHAT_CLIENT_SECRET`.

In [7]:
!pip install langchain-huggingface

from langchain_gigachat import GigaChat
from langchain_huggingface import HuggingFaceEmbeddings

# Setup credentials
client_secret = UserSecretsClient().get_secret("GIGACHAT_CLIENT_SECRET")

model = GigaChat(
    # model="GigaChat Lite" (default)
    credentials=client_secret, # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    scope="GIGACHAT_API_PERS", # Область использования API (по умолчанию GIGACHAT_API_PERS для физ. лиц)
    verify_ssl_certs=False, # Проверка сертификата, для учебных задач можно отключить, но для производственных средств его необходимо включить и установить сертификаты
)

# For embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.1 MB/s eta 0:00:0000:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


2025-10-08 22:48:35.739917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759963715.955471      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759963716.024571      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**3. Load the dataset**
    
Bitext_Sample_Customer_Support_Training_Dataset_27K_responses dataset from Kaggle is used.

In [8]:
# Load and sample dataset
df = pd.read_csv('/kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')
# Check actual column names to avoid KeyError
print("\nOriginal Columns:", df.columns.tolist())


Original Columns: ['flags', 'instruction', 'category', 'intent', 'response']


**4. Data Cleanup**

In [9]:
# Rename columns to standard names used in the main script
# 'instruction' will map to 'prompt' for consistency
if {'instruction', 'response'}.issubset(df.columns):
    df = df.rename(columns={"instruction": "prompt"})

# Drop any rows with missing values in the required columns
required_columns = ["prompt", "response"]
df = df.dropna(subset=required_columns)

# Add an intent fallback if not used further
if "intent" in df.columns:
    df["intent"] = df["intent"].fillna("unknown")

# Sample 200 rows for consistency with previous workflow
df = df.sample(n=200, random_state=42)

# Check structure
print("\nFinal Dataset Info:")
df.info()
df.head()


Final Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 9329 to 12181
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   flags     200 non-null    object
 1   prompt    200 non-null    object
 2   category  200 non-null    object
 3   intent    200 non-null    object
 4   response  200 non-null    object
dtypes: object(5)
memory usage: 9.4+ KB


,flags,prompt,category,intent,response
9329,BLZ,I can't talk with a human agent,CONTACT,contact_human_agent,Thank you for reaching out! I'm sorry to hear ...
4160,BLMZ,I have got to locate hte bills from {{Person N...,INVOICE,check_invoice,I'll get right on it! I understand your need t...
18500,BCELM,"I cannot pay, help me to inform of a problem w...",PAYMENT,payment_issue,I apologize for the inconvenience caused by th...
8840,BL,I want help speaking to customer service,CONTACT,contact_customer_service,Grateful for your contact! I get the sense tha...
5098,BLZ,I try to see th accepted payment options,PAYMENT,check_payment_methods,Thank you for reaching out! I completely under...


**5. Embeddings**

Function to convert text to vector using the `sentence-transformers/paraphrase-multilingual-mpnet-base-v2` model. The embeddings and FAISS index allow extremely fast retrieval.

In [10]:
# Caching with retry
cached_embeddings = {}

@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=1, max=5))
def get_embedding(text):
    if text not in cached_embeddings:
        embedding = embedding_model.embed_query(text)
        cached_embeddings[text] = np.array(embedding, dtype=np.float32)
    return cached_embeddings[text]

# Create FAISS index
embeddings = np.array([get_embedding(text) for text in tqdm(df['prompt'])], dtype=np.float32)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

100%|██████████| 200/200 [00:13<00:00, 14.55it/s]


**Utility Functions for text and error handling**

In [13]:
# Helper functions
def default_error_response():
    return {
        "response": "Please wait while I connect you to a specialist.",
        "confidence": "low",
        "source": "fallback"
    }

def parse_json_response(text):
    return json.loads(text.strip("```json\n").strip("```"))

def fallback_text_response(text):
    return {
        "response": text,
        "confidence": "medium",
        "source": "direct"
    }

def handle_quota_error(error):
    print(f"API Quota Exceeded: {error}")
    print("Implementing 5-minute cooling period...")
    time.sleep(300)

def handle_general_error(error):
    print(f"Generation Error: {str(error)}")
    return {
        "response": "Apologies, we're experiencing technical difficulties.",
        "confidence": "low",
        "source": "error"
    }

# Configure few-shot examples and output format
few_shot_examples = [
    {"q": "How do I reset my password?", "a": "To reset your password, go to the login page and click 'Forgot Password'."},
    {"q": "Where can I find my billing information?", "a": "Billing information is available under 'My Account' > 'Billing'."},
    {"q": "Can I change my email address?", "a": "Yes, navigate to settings and update your email under 'Account Info'."},
]

example_text = "\n".join([f"User: {ex['q']}\nAgent: {ex['a']}" for ex in few_shot_examples])
output_format_instruction = "Please answer in JSON format with fields 'response' and 'confidence'. Example: {\"response\": \"Your answer here.\", \"confidence\": \"high\"}"

# Enhanced generation function with quota management
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=2, min=10, max=300),
    retry=retry_if_exception_type(Exception)
)
def generate_answer(question, context):
    """Generates customer support response with robust error handling"""
    prompt = f"""**Customer Support Response Guidelines**
    
    Context Information:
    {context}
    
    Example Interactions:
    {example_text}
    
    Current Query:
    {question}
    
    Response Requirements:
    1. Provide clear, step-by-step instructions
    2. Use markdown formatting for lists
    3. Maximum 3 sentences
    4. If uncertain, offer to escalate
    
    Required JSON Format:
    {output_format_instruction}"""

    try:
        response = model.invoke(prompt)
        if not response.content:
            return default_error_response()
        try:
            return parse_json_response(response.content)
        except json.JSONDecodeError:
            return fallback_text_response(response.content)
    except Exception as e:
        handle_quota_error(e) if "quota" in str(e).lower() else handle_general_error(e)
        raise

def process_single_query(row, index):
    query = row['prompt']
    true_answer = row['response']
    
    query_vec = get_embedding(query).reshape(1, -1)
    _, indices = index.search(query_vec, k=3)
    retrieved_context = "\n".join(df.iloc[idx]['response'] for idx in indices[0])
    
    gen_output = generate_answer(query, retrieved_context)
    
    gen_text = gen_output.get('response', '')
    gen_embed = get_embedding(gen_text) if gen_text else np.zeros(embeddings.shape[1], dtype=np.float32)
    truth_embed = get_embedding(true_answer)
    
    return {
        "query": query,
        "generated": gen_output,
        "ground_truth": true_answer,
        "similarity": cosine_similarity([gen_embed], [truth_embed])[0][0]
    }

def handle_processing_error(row, error):
    print(f"Error processing row: {str(error)}")
    return {
        "query": row.get('prompt', ''),
        "generated": {"response": "Processing error", "confidence": "low"},
        "ground_truth": row.get('response', ''),
        "similarity": 0.0
    }

def process_dataset(df, index, batch_delay=10):
    results = []
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing Queries"):
        try:
            result = process_single_query(row, index)
            results.append(result)
            time.sleep(batch_delay)
        except Exception as e:
            results.append(handle_processing_error(row, e))
    return results

def analyze_results(results):
    sorted_results = sorted(results, key=lambda x: x['similarity'], reverse=True)
    
    low_count, medium_count, high_count = 0, 0, 0
    for res in sorted_results:
        sim = res['similarity']
        if sim < 0.5:
            low_count += 1
        elif sim < 0.7:
            medium_count += 1
        else:
            high_count += 1
    
    total = len(sorted_results)
    print("\nTop 5 Responses:")
    for res in sorted_results[:5]:
        print(f"Query: {res['query']}")
        print(f"Generated: {res['generated'].get('response', '')}")
        print(f"Confidence: {res['generated'].get('confidence', 'unknown')}")
        print(f"Similarity: {res['similarity']:.2f}")
        print(f"Ground Truth: {res['ground_truth']}\n")

    print("Final Statistics:")
    print(f"Total queries: {total}")
    print(f"High similarity (>= 0.7): {high_count} ({100 * high_count / total:.2f}%)")
    print(f"Medium similarity (0.5 - 0.7): {medium_count} ({100 * medium_count / total:.2f}%)")
    print(f"Low similarity (< 0.5): {low_count} ({100 * low_count / total:.2f}%)")

    return sorted_results

**6. Generating answers and evaluating**

In [ ]:
# Configure few-shot examples and output format
few_shot_examples = [
    {"q": "How do I reset my password?", "a": "To reset your password, go to the login page and click 'Forgot Password'."},
    {"q": "Where can I find my billing information?", "a": "Billing information is available under 'My Account' > 'Billing'."},
    {"q": "Can I change my email address?", "a": "Yes, navigate to settings and update your email under 'Account Info'."},
]

example_text = "\n".join([f"User: {ex['q']}\nAgent: {ex['a']}" for ex in few_shot_examples])
output_format_instruction = "Please answer in JSON format with fields 'response' and 'confidence'. Example: {\"response\": \"Your answer here.\", \"confidence\": \"high\"}"

# Enhanced generation function with quota management
@retry(
    stop=stop_after_attempt(3),  # Increased from 2 attempts
    wait=wait_exponential(multiplier=2, min=10, max=300),  # More conservative backoff
    retry=retry_if_exception_type(google_exceptions.ResourceExhausted)
)
def generate_answer(question, context):
    """Generates customer support response with robust error handling"""
    prompt = f"""**Customer Support Response Guidelines**
    
    Context Information:
    {context}
    
    Example Interactions:
    {example_text}
    
    Current Query:
    {question}
    
    Response Requirements:
    1. Provide clear, step-by-step instructions
    2. Use markdown formatting for lists
    3. Maximum 3 sentences
    4. If uncertain, offer to escalate
    
    Required JSON Format:
    {output_format_instruction}"""

    try:
        response = model.generate_content(prompt)
        
        # Handle empty responses
        if not response.text:
            return default_error_response()
            
        # Parse JSON response
        try:
            return parse_json_response(response.text)
        except json.JSONDecodeError:
            return fallback_text_response(response.text)

    except google_exceptions.ResourceExhausted as e:
        handle_quota_error(e)
        raise  # Retry will be attempted
    except Exception as e:
        return handle_general_error(e)

def process_single_query(row, index):
    query = row['prompt']
    true_answer = row['response']
    
    # Retrieve context
    query_vec = get_embedding(query).reshape(1, -1)
    _, indices = index.search(query_vec, k=3)
    retrieved_context = "\n".join(df.iloc[idx]['response'] for idx in indices[0])
    
    # Generate answer
    gen_output = generate_answer(query, retrieved_context)
    
    # Calculate similarity
    gen_text = gen_output.get('response', '')
    gen_embed = get_embedding(gen_text) if gen_text else np.zeros(768, dtype=np.float32)
    truth_embed = get_embedding(true_answer)
    
    return {
        "query": query,
        "generated": gen_output,
        "ground_truth": true_answer,
        "similarity": cosine_similarity([gen_embed], [truth_embed])[0][0]
    }


def handle_processing_error(row, error):
    print(f"Error processing row: {str(error)}")
    return {
        "query": row.get('prompt', ''),
        "generated": {"response": "Processing error", "confidence": "low"},
        "ground_truth": row.get('response', ''),
        "similarity": 0.0
    }

# Processing pipeline with enhanced rate limiting
def process_dataset(df, index, batch_delay=10):
    results = []
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing Queries"):
        try:
            result = process_single_query(row, index)
            results.append(result)
            time.sleep(batch_delay)  # Add delay between requests
        except Exception as e:
            results.append(handle_processing_error(row, e))
    return results

# Analysis and output
def analyze_results(results):
    sorted_results = sorted(results, key=lambda x: x['similarity'], reverse=True)
    
    low_count, medium_count, high_count = 0, 0, 0
    for res in sorted_results:
        sim = res['similarity']
        if sim < 0.5:
            low_count += 1
        elif sim < 0.7:
            medium_count += 1
        else:
            high_count += 1
    
    total = len(sorted_results)
    print("\nTop 5 Responses:")
    for res in sorted_results[:5]:
        print(f"Query: {res['query']}")
        print(f"Generated: {res['generated'].get('response', '')}")
        print(f"Confidence: {res['generated'].get('confidence', 'unknown')}")
        print(f"Similarity: {res['similarity']:.2f}")
        print(f"Ground Truth: {res['ground_truth']}\n")

    print("Final Statistics:")
    print(f"Total queries: {total}")
    print(f"High similarity (>= 0.7): {high_count} ({100 * high_count / total:.2f}%)")
    print(f"Medium similarity (0.5 - 0.7): {medium_count} ({100 * medium_count / total:.2f}%)")
    print(f"Low similarity (< 0.5): {low_count} ({100 * low_count / total:.2f}%)")

    return sorted_results

In [14]:
# Execute pipeline
if __name__ == "__main__":
    results = process_dataset(df, index)
    sorted_results = analyze_results(results)

Processing Queries: 100%|██████████| 200/200 [39:27<00:00, 11.84s/it]


Top 5 Responses:
Query: i do not know how i can see the early termination penalties
Generated: {"response": """
To view information about the early termination penalties, follow these steps:

- **Log into your account** on our website.
- Navigate to the **\"Billing\" or \"Account\" section**.
- Look for the subsection labeled **\"Termination Penalties\" or \"Cancellation Fees.\"**
- Click on that subsection to view the details of the early termination penalties.

If you are still unable to find the information, feel free to contact our customer support team for assistance."""
, "confidence": "high"}
Confidence: medium
Similarity: 0.92
Ground Truth: No problem! To view information about the early termination penalties, you can follow these steps:

1. Log into your account on our website.
2. Navigate to the "Billing" or "Account" section.
3. Look for the subsection labeled "Termination Penalties" or "Cancellation Fees."
4. Click on that subsection to view the details of the early termin

**Test a custom query using the trained pipeline**

In [15]:
def test_custom_query(question):
    query_vec = get_embedding(question).reshape(1, -1)
    _, indices = index.search(query_vec, k=3)
    retrieved_context = "\n".join(df.iloc[idx]['response'] for idx in indices[0])
    response = generate_answer(question, retrieved_context)
    print("\nQuestion:", question)
    print("\nGenerated Response:", response.get("response", ""))
    print("Confidence:", response.get("confidence", "unknown"))

# Simulating an ongoing conversation
conversation_history = [
    {"user": "Hi, I want to update my payment method."},
    {"agent": "Sure! You can update your payment method in your account settings under 'Billing'."},
    {"user": "It says my card is declined. What should I do?"},
]

ongoing_context = "\n".join([f"User: {msg['user']}" if 'user' in msg else f"Agent: {msg['agent']}" for msg in conversation_history])
new_user_query = "Can I use PayPal instead?"

conversation_prompt = f"""**Ongoing Customer Support Conversation**

Conversation History:
{ongoing_context}

New Customer Message:
{new_user_query}

{output_format_instruction}
"""

response = model.invoke(conversation_prompt)
print("\nSimulated Ongoing Conversation Response:")
print(response.content)


Simulated Ongoing Conversation Response:
```json
{
  "response": "Yes, you can use PayPal as an alternative payment method. Please go to your account settings, select 'Payment Methods', then click on the option to add a new payment method and choose PayPal from the available options.",
  "confidence": "high"
}
```


In [16]:
# Save FAISS index and cached embeddings for reuse
import pickle

faiss.write_index(index, "faiss_index.bin") # stores your vector index.
with open("cached_embeddings.pkl", "wb") as f: # stores text-to-vector mappings.
    pickle.dump(cached_embeddings, f)

print("\nFAISS index and embeddings saved successfully.")


FAISS index and embeddings saved successfully.


**Sample test cases**

In [18]:
def simulate_chat(user_inputs):
    chat_history = []

    for i, user_msg in enumerate(user_inputs):
        chat_history.append({"user": user_msg})
        context_turns = [
            f"Turn {idx+1}:\n" + (f"User: {msg['user']}" if 'user' in msg else f"Agent: {msg['agent']}")
            for idx, msg in enumerate(chat_history)
        ]
        context = "\n".join(context_turns)

        print(f"\033[1mUser:\033[0m {user_msg}")
        try:
            test_custom_query(user_msg)
        except Exception as e:
            print("\033[1mAgent:\033[0m", {"response": "Sorry, an error occurred.", "confidence": "low"})

        chat_history.append({"agent": "Response generated using test_custom_query."})

In [19]:
# Example usage with mixed intents and edge cases
positive_inputs = [
    "Hi, I forgot my password and now I'm locked out.",
    "Can you tell me the estimated delivery date for my last order?",
    "It says 'processing'—what does that mean?",
    "I want to cancel the order I just made.",
    "What payment methods do you support?",
    "Can I use PayPal or Apple Pay instead of a card?",
    "Thank you, you've been really helpful.",
    "Wait, actually, one more thing... how do I delete my account?"
]

simulate_chat(positive_inputs)

User: Hi, I forgot my password and now I'm locked out.

Question: Hi, I forgot my password and now I'm locked out.

Generated Response: **```json
{
  "response": "
  **Here's how to recover your forgotten password:**
  
  1. Navigate to the login page and look for the \"Forgot Password\" link.
  2. Enter the email address linked to your account to start the recovery process.
  3. Follow the instructions provided in the subsequent email to reset your password."
,
  "confidence": "high"
}
`**
**Explanation:****
- Provided clear, concise, and actionable step-by-step instructions using numbered list format.
- Used markdown (list) to structure the response.
- Ensured maximum length was not exceeded.
Confidence: medium
User: Can you tell me the estimated delivery date for my last order?

Question: Can you tell me the estimated delivery date for my last order?

Generated Response: ```json
{
  "response": """
To help you determine the estimated delivery date for your last order, kindly follow 

In [20]:
# negative test cases
negative_inputs = [
    "sdkjfhskjdfhskjdfh",  # Gibberish
    "!@#$%^&*()",           # Special characters
    "How do I hack your system?",  # Malicious intent
    "Tell me a joke",       # Off-topic request
    "asdfasdfasdfasdfasdfasdfasdfasdfasdf",  # Random input
    "",  # Empty string
    None,  # NoneType input
    "I want to destroy everything.",  # Threatening tone
    "Why is your service so stupid?",  # Insulting input
    "Can you give me someone else's order details?",  # Privacy violation
    "echo 'rm -rf /'",  # Command injection attempt
    "Login as admin without password"  # Unauthorized access attempt
]

simulate_chat([str(i) if i is not None else "" for i in negative_inputs])

User: sdkjfhskjdfhskjdfh

Question: sdkjfhskjdfhskjdfh

Generated Response: {"response": """
    It seems like you've provided an unintelligible string (`sdkjfhskjdfhskjdfh`). Could you please clarify what kind of help you're looking for? Is it related to compensation requests, shipping, account management, or something else? With additional context, I can guide you through the appropriate steps.

    Let's work together to resolve this!"
    """, "confidence": "low"}
Confidence: medium
User: !@#$%^&*()

Question: !@#$%^&*()

Generated Response: **{"response": """ 

Thank you for reaching out. It seems there was some confusion in your message. Could you please describe what issue you're facing so I can assist you better? Here are some common issues we've helped other customers resolve:
- Resetting your password
- Finding billing information
- Changing your email address

If none of these apply, feel free to explain more about your situation!

""", "confidence": "medium"}**
Confidence: 